## To run the code, make sure that you have the correct file structure:

Create a folder where you would like to hold all of the data for analysis, which includes the images, and metadata. Keep track of this file path, you will be asked to provide it to perform the analysis.
If you like fill the provideded template metadat '.xlxs' with the exact same name as the image file. For each image file (scan) you need to provide an excel file with same name. 
All the analysis data will be saved on the metadata '.xlxs' file as different sheets.



#### Load Libraries

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.util import crop
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import measure
from skimage.morphology import closing, square, remove_small_objects
#from tifffile import imsave
from tkinter import*
from tkinter import filedialog
import os
import glob
import os, sys
import os, errno
import shutil, os


import time
import skimage
from skimage import io
from skimage import filters
from skimage import feature
from skimage import morphology
from skimage import exposure
import matplotlib as mpl
import scipy.stats as stats
from scipy import ndimage as ndi
import pathlib as plb 
import csv
%matplotlib inline

## Determine the files directoty

In [3]:
def directoryy():
    root = Tk()
    root.withdraw()
    current_directory = filedialog.askdirectory()
    path = os.path.join(current_directory)
    %gui tk
    print(path)
 #   path="/Users/erezaei/Desktop/Neuroplant-ER_EF_GUI/MY_Test"
    os.chdir(path)
    extension = 'tif'
    current_di = glob.glob('*.{}'.format(extension))
    #print(current_dir)

    path_img = path+'/Images'
    try:
        os.makedirs(path_img)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    path_rslt = path+'/automated_results'
    try:
        os.makedirs(path_rslt)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    for f in current_di:
        shutil.move(f, path_img)

    if not current_di:
        os.chdir(path_img)
        current_di = glob.glob('*.{}'.format(extension))
        print(current_di)
        
        return current_di,path_rslt,path_img

In [4]:
def loopOver(current_di,path_rslt,path_img):
    for File_nam in current_di:
        
        os.chdir(path_img)
        image=np.invert(imread(File_nam))
        rows = []
        # apply threshold
        thresh = threshold_otsu(image)
        bw = closing(image > thresh, square(10))

        # remove artifacts connected to image border
        cleared = remove_small_objects(clear_border(bw), 1000000)

        # label image regions
        label_image = label(cleared)
        image_label_overlay = label2rgb(label_image, image=image, bg_label=0)
        print(File_nam)
        
        img_prop(label_image,image,File_nam,path_rslt)  

In [5]:
def img_prop(label_image,image,File_nam,path_rslt):
    props = regionprops_table(label_image, properties=('label','centroid', 'bbox', 'area'))
    dff=pd.DataFrame(props)
    df=dff.sort_values(by=['bbox-1'])
    df.reset_index(drop=True, inplace=True)
    print('test1')
    # Sort the plates into the left and right
    MinCol=1.1*list(df.items())[4][1][7]
    mask = df['bbox-1'] > MinCol
    df_r = df[mask]
    df_l = df[~mask]

    ## Split the right plates to upper (#1) and lower (#4)
    dffr=df_r.sort_values(by=['bbox-0'])
    dffr.reset_index(drop=True, inplace=True)
    MinCol1=1.05*list(dffr.items())[3][1][3]
    mask = df_r['bbox-0'] > MinCol1
    dff4 = df_r[mask]
    dff1 = df_r[~mask]
    print('test2')
    ### Sort the wells on each plate
    df1=dff1.sort_values(by=['bbox-0'])
    df4=dff4.sort_values(by=['bbox-0'])
    df4.reset_index(drop=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)
    print('test3')
    ## Split the left plates to upper (#2) and lower (#3)
    dffl=df_l.sort_values(by=['bbox-0'])
    dffl.reset_index(drop=True, inplace=True)
    MinCol2=1.05*list(dffl.items())[3][1][3]
    mask = df_l['bbox-0'] > MinCol2
    dff3 = df_l[mask]
    dff2 = df_l[~mask]

    ### Sort the wells on each plate
    df2=dff2.sort_values(by=['bbox-0'])
    df3=dff3.sort_values(by=['bbox-0'])
    df3.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    ## Update the label of each well
    new_label_1 = pd.Series(['1P', '1Q','1R','1S'], name='label', index=[0,1,2,3])
    df1.update(new_label_1)

    new_label_2 = pd.Series(['2P', '2Q','2R','2S'], name='label', index=[0,1,2,3])
    df2.update(new_label_2)

    new_label_3 = pd.Series(['3P', '3Q','3R','3S'], name='label', index=[0,1,2,3])
    df3.update(new_label_3)

    new_label_4 = pd.Series(['4P', '4Q','4R','4S'], name='label', index=[0,1,2,3])
    df4.update(new_label_4)
    print('test4')
    ### Append the dataframes
    df_f=df1.append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True)
    df_f["Chemotaxis"] = np.nan
    print(df_f)
    loopWell(df_f,image,File_nam,path_rslt)
    

### Find worms

In [1]:
def loopWell(df_f,image,File_nam,path_rslt):
    
    for WellNo in range(len(df_f)):
#     fin_image = image[ df_f[][]:Lower_boundary , Left_boundary:Right_boundary ]
        fin_image = image[ df_f['bbox-0'][WellNo]:df_f['bbox-2'][WellNo] , df_f['bbox-1'][WellNo]:df_f['bbox-3'][WellNo] ]
        
        #rows = []
        # apply threshold
        
        #thres = threshold_otsu(fin_image)
        #bww = closing(fin_image > thres, square(10))
        
        thresh = skimage.filters.threshold_li(fin_image)
        binarized = fin_image > thresh

        # remove artifacts connected to image border
        #cleare = remove_small_objects(clear_border(bww), 90)
        
                ## Find features in binary image
        labeled_array, num_features = ndi.label(binarized)
        #all_regions = measure.regionprops(label_image=labeled_array, intensity_image=fin_image,
        #                                  coordinates='rc')
        props_worm = regionprops_table(label_image=labeled_array, properties=('label','centroid', 'area'))
        worms=pd.DataFrame(props_worm)
        
        # label image regions
        #label_image = label(cleare)
        #image_label_overlayy = label2rgb(label_image, image=fin_image, bg_label=0)
        filt_worm=worms[worms['area']<2000]
        filtered_worm=filt_worm[filt_worm['area']>10]
        print(df_f['label'][WellNo])

        #w_df=pd.DataFrame(all_regions)
        #filtered_regions = []
        #for region in all_regions:
          #  area = region.area

           # if area >= 90 and area <=1000 and region.major_axis_length < 150:
          #      filtered_regions.append(region)
        #print(worms)
        print(filtered_worm)
        calc_chemotaxis_index(df_f,filtered_worm,File_nam,path_rslt,WellNo)
        save_worm_locations(df_f,filtered_worm,File_nam,path_rslt,WellNo)
    save_Summary(df_f,File_nam)

SyntaxError: invalid syntax (<ipython-input-1-9ae5ba63dad8>, line 3)

In [7]:
def calc_chemotaxis_index(df_f,filtered_worm,File_nam,path_rslt,WellNo):
    
    ## Assign features to zones of the plate
    left_area_boundary = df_f['bbox-0'][WellNo]+4/9*(df_f['bbox-2'][WellNo]-df_f['bbox-0'][WellNo])#int(3424*4/9) #8000
    right_area_boundary = df_f['bbox-0'][WellNo]+5/9*(df_f['bbox-2'][WellNo]-df_f['bbox-0'][WellNo])#int(3424*5/9)

    left_side_worms = filtered_worm[filtered_worm['centroid-1'] <= left_area_boundary]
    middle_w = filtered_worm[filtered_worm['centroid-1'] < right_area_boundary]
    middle_worms = middle_w[middle_w['centroid-1'] >left_area_boundary]
    right_side_worms = filtered_worm[filtered_worm['centroid-1'] >= right_area_boundary]

    ## Calculate chemotaxis index
    worms_in_left_region = len(left_side_worms)
    worms_in_middle_region = len(middle_worms)
    worms_in_right_region = len(right_side_worms)
    total_worms_found = len(filtered_worm)
    #print('Left; ' + str(worms_in_left_region) + '   Right: ' + str(worms_in_right_region))

    chemotaxis_index = ((worms_in_left_region - worms_in_right_region) 
                        / (worms_in_left_region + worms_in_right_region))
    df_f['Chemotaxis'][WellNo]=chemotaxis_index
    print(df_f)

### Save worm locations

In [8]:
def save_worm_locations(df_f,filtered_worm,File_nam,path_rslt,WellNo):
    #os.chdir(path_rslt)
    filename = File_nam[0:-4]+'.xlsx'
    with pd.ExcelWriter(filename, mode='a') as writer:  
        filtered_worm.to_excel(writer, sheet_name=df_f['label'][WellNo])

In [9]:
def save_Summary(df_f,File_nam):
    #os.chdir(path_rslt)
    filename = File_nam[0:-4]+'.xlsx'
    with pd.ExcelWriter(filename, mode='a') as writer:  
        df_f.to_excel(writer, sheet_name='Sum')

In [10]:
def check_num_images():
    current_di,path_rslt,path_img= directoryy()
    loopOver(current_di,path_rslt,path_img)

In [14]:
check_num_images()

/Users/emilyfryer/Documents/Current_analysis/Images
[]
